In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
from pandas.core.dtypes.common import is_numeric_dtype
from sklearn.preprocessing import MinMaxScaler

DATA_FOLDER = 'dataset'
DATASET = os.path.join(DATA_FOLDER, 'preprocessed_df.csv')

df = pd.read_csv(DATASET)
numerical_attr_list = [col for col in df.columns if is_numeric_dtype(df[col])]

normalized_df = MinMaxScaler().fit_transform(df[numerical_attr_list].values)

In [12]:
def plot_outliers_in_embedding(embedding):
    fig, axes = plt.subplots(2,2, sharex=True, sharey=True)
    axabod, axlof, axisfo, _ = axes.flatten()
    methods = [s.format(cat="vc") for s in ["is_outlier_{cat}_ABOD",
                                            "is_outlier_{cat}_LOF", 
                                            "is_outlier_{cat}_ISFO"]
                ]
    for ax, method in zip([axabod, axlof, axisfo], ["is_outlier"]):
        ax.scatter(*embedding.T, c=df[method], cmap="plasma")
    return fig, axes

In [ ]:
from sklearn.decomposition import PCA
res = PCA(n_components=2)
embedding = res.fit_transform(normalized_df)
plot_outliers_in_embedding(embedding)

In [ ]:
import umap
reducer = umap.UMAP(n_components=2, n_neighbors=6)
embedding = reducer.fit_transform(normalized_df)
plot_outliers_in_embedding(embedding)

In [ ]:
from sklearn.manifold import Isomap
reducer = Isomap(n_components=2, n_neighbors=6)
embedding = reducer.fit_transform(normalized_df)